In [1]:
## Import Section

import cv2
import numpy as np
import matplotlib.pyplot as plt
import skimage as sk
import skimage.io as skio
from gif_maker import make_gif
from math import sqrt

import itertools
import sys
import os

In [2]:
## Defining Basic Functions

def plotgray(im, cmap='gray'):
    fig = plt.figure()
    plt.imshow(im, cmap=cmap)
    plt.show()

def plot3ch(im):
    fig = plt.figure()
    plt.imshow(im)
    plt.show()

def resize(im, scale):
    h, w = im.shape[:2]
    return sk.transform.resize(im, (int(h*scale), int(w*scale)))

# shift an image `tj` to the right, `ti` down
def shift(im, ti, tj):
    return np.roll(im, (ti, tj), axis=(0, 1))

# translate an image `tx` to the right, `ty` down
def translate(im, tx, ty):
    h, w = im.shape[:2]
    mat = translation_matrix = np.array([
        [1, 0, tx],
        [0, 1, ty]
    ], dtype=np.float32)
    return cv2.warpAffine(im, mat, (w, h))

def normalize_clip(im):
    return np.minimum(np.maximum(im, 0), 1)

def normalize(im):
    return (im-np.min(im))/(np.max(im)-np.min(im))

def rgb2gray(rgb):
    return rgb.dot([0.2989, 0.5870, 0.1140])

# crop the borders of a given image by `ratio` percent
def crop_with_percent(im, ratio=0.4):
    h, w = im.shape[:2]
    return im[int(h*(ratio/2)):int(h*(1-ratio/2)), int(w*(ratio/2)):int(w*(1-ratio/2))]

In [3]:
# achieving both Depth Refocusing and Aperture Adjustment
def refocusing(ims, us, vs, idx, C, aperture_size=1.): # Note that aperture size is relative, and default is full
    
    im = np.zeros(ims[0].shape)
    uc, vc = us[idx[8, 8]], vs[idx[8, 8]]
    
    # compute the maximum absolute aperture size
    mx = 0.
    for i in range(0, 17):
        for j in range(0, 17):
            ui, vi = us[idx[i, j]], vs[idx[i, j]]
            mx = max((vi-vc)**2+(ui-uc)**2, mx)
    mx = sqrt(mx)
    
    # loop over the aperture
    cnt = 0
    for i in range(0, 17):
        for j in range(0, 17):
            imi, ui, vi = ims[idx[i, j]], us[idx[i, j]], vs[idx[i, j]]
            if sqrt((vi-vc)**2+(ui-uc)**2)/mx > aperture_size:
                continue
            # shift the sub-aperture images and average them
            im += translate(imi, C*(vi-vc), C*(ui-uc))
            cnt += 1

    im /= cnt
    return im

In [16]:
## sample 1

input_dir = "./sample1"
output_dir = "./output/sample1"

# load the images
ims, us, vs = [], [], []
idx, tot = np.ndarray((17, 17), dtype=int), 0
for file_name in os.listdir(input_dir):
    im = sk.img_as_float(skio.imread(os.path.join(input_dir, file_name)))
    file_rt_name, _ = os.path.splitext(file_name)
    rdl = file_rt_name.strip().split('_')
    r, c, u, v = int(rdl[1]), int(rdl[2]), float(rdl[3]), float(rdl[4])
    ims.append(im); us.append(u); vs.append(v)
    idx[r, c] = tot
    tot += 1

In [15]:
# directly averaging all images
im = refocusing(ims, us, vs, idx, 0.)
plt.imsave(os.path.join(output_dir, "sum.jpg"), im)

In [18]:
# refocusing
num_frames, frame_folder_name = 0, "frame_folder_pt1"
os.makedirs(os.path.join(output_dir, frame_folder_name), exist_ok=True)
for i, C in enumerate(np.arange(-0.2, 0.7, 0.02)):
    im = refocusing(ims, us, vs, idx, C)
    plt.imsave(os.path.join(output_dir, frame_folder_name, f"{i}.jpg"), resize(im, 0.3))
    num_frames += 1
make_gif(output_dir, frame_folder_name, num_frames, "pt1.gif", duration=60)

In [6]:
# aperture adjustment
num_frames, frame_folder_name = 0, "frame_folder_pt2"
os.makedirs(os.path.join(output_dir, frame_folder_name), exist_ok=True)
for i, r in enumerate(np.arange(0.1, 1.05, 0.05)):
    im = refocusing(ims, us, vs, idx, 0.1, r)
    plt.imsave(os.path.join(output_dir, frame_folder_name, f"{i}.jpg"), resize(im, 0.5))
    num_frames += 1
make_gif(output_dir, frame_folder_name, num_frames, "pt2.gif", duration=100)

In [7]:
## sample 2

input_dir = "./sample2"
output_dir = "./output/sample2"

# load the images
ims, us, vs = [], [], []
idx, tot = np.ndarray((17, 17), dtype=int), 0
for file_name in os.listdir(input_dir):
    im = sk.img_as_float(skio.imread(os.path.join(input_dir, file_name)))
    file_rt_name, _ = os.path.splitext(file_name)
    rdl = file_rt_name.strip().split('_')
    r, c, u, v = int(rdl[1]), int(rdl[2]), float(rdl[3]), float(rdl[4])
    ims.append(im); us.append(u); vs.append(v)
    idx[r, c] = tot
    tot += 1

In [10]:
# refocusing
num_frames, frame_folder_name = 0, "frame_folder_pt1"
os.makedirs(os.path.join(output_dir, frame_folder_name), exist_ok=True)
for i, C in enumerate(np.arange(-0.6, 0.1, 0.02)):
    im = refocusing(ims, us, vs, idx, C)
    plt.imsave(os.path.join(output_dir, frame_folder_name, f"{i}.jpg"), resize(im, 1.))
    num_frames += 1
make_gif(output_dir, frame_folder_name, num_frames, "pt1.gif", duration=60)

In [12]:
# aperture adjustment
num_frames, frame_folder_name = 0, "frame_folder_pt2"
os.makedirs(os.path.join(output_dir, frame_folder_name), exist_ok=True)
for i, r in enumerate(np.arange(0.1, 1.05, 0.05)):
    im = refocusing(ims, us, vs, idx, -0.1, r)
    plt.imsave(os.path.join(output_dir, frame_folder_name, f"{i}.jpg"), resize(im, 1.))
    num_frames += 1
make_gif(output_dir, frame_folder_name, num_frames, "pt2.gif", duration=100)